Ordering and Delivery to Google Earth Engine (GEE)

Based off of the client's example (https://github.com/planetlabs/planet-client-python/blob/main/examples/orders_create_and_download_multiple_orders.py)

In [3]:
import asyncio
import os

import planet

Authetication

In [ ]:
# API Keys stored as an env variable
PLANET_API_KEY = os.getenv('PL_API_KEY')
GEE_API_KEY = os.getenv('GEE_API_KEY')

Define order

In [5]:
DOWNLOAD_DIR = os.getenv('TEST_DOWNLOAD_DIR', '.')

iowa_aoi = {
    "type":
    "Polygon",
    "coordinates": [[[-91.198465, 42.893071], [-91.121931, 42.893071],
                     [-91.121931, 42.946205], [-91.198465, 42.946205],
                     [-91.198465, 42.893071]]]
}

iowa_images = ['20200925_161029_69_2223', '20200925_161027_48_2223']

Define cloud delivery location

In [6]:
google_cloud_storage_config = {
        "delivery": {
            "google_cloud_storage": {
                "project": "planet-earthengine-staging",
                "collection": "gee-integration-testing",
                "credentials": GEE_API_KEY
            }
    }
}
delivery_config = planet.order_request.delivery(
    cloud_config=planet.order_request.google_earth_engine(project='planet-earthengine-staging', collection='gee-integration-testing')
)

Build order request

In [7]:
iowa_order = planet.order_request.build_request(
    'iowa_order',
    [planet.order_request.product(iowa_images, 'analytic', 'PSScene4Band')],
    delivery=planet.order_request.delivery(cloud_config=delivery_config),
    tools=[planet.order_request.clip_tool(iowa_aoi)])

print(iowa_order)

{'name': 'iowa_order', 'products': [{'item_ids': ['20200925_161029_69_2223', '20200925_161027_48_2223'], 'item_type': 'PSScene4Band', 'product_bundle': 'analytic'}], 'delivery': {'google_earth_engine': {'project': 'planet-earthengine-staging', 'collection': 'gee-integration-testing'}}, 'tools': [{'clip': {'aoi': {'type': 'Polygon', 'coordinates': [[[-91.198465, 42.893071], [-91.121931, 42.893071], [-91.121931, 42.946205], [-91.198465, 42.946205], [-91.198465, 42.893071]]]}}}]}


Define create and download order functions

In [8]:
async def create_and_download(order_detail, directory, client):
    with planet.reporting.StateBar(state='creating') as reporter:
        # create
        order = await client.create_order(order_detail)
        reporter.update(state='created', order_id=order['id'])
        # wait for completion
        await client.wait(order['id'], callback=reporter.update_state)

    order_details = await client.get_order(order_id=order['id'])
    return order_details

Create and download order

In [9]:
async with planet.Session() as ps:
    client = planet.OrdersClient(ps)
    order_details = await create_and_download(iowa_order, DOWNLOAD_DIR, client)

07:42 - order 4e8e46ee-7d62-425c-b9c3-2bc6a48a9c36 - state: success


In [10]:
print(order_details['id'])
print(order_details['_links']['results'][0])

4e8e46ee-7d62-425c-b9c3-2bc6a48a9c36
{'delivery': 'success', 'expires_at': '2022-10-21T18:36:12.110Z', 'location': 'https://api.planet.com/compute/ops/download/?token=eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJleHAiOjE2NjYzNzczNzIsInN1YiI6InZwdTN2WitMeTdjdHdhN3JPR1ZmYkUxNlcrdjc5Y0RZQ0FsZUYrN1N4S0dFT1RjbjBVOTJIRmxEZVhzNlA3RDlVeCtmb1FLdkttVjlzWldmYnFSS213PT0iLCJ0b2tlbl90eXBlIjoiZG93bmxvYWQtYXNzZXQtc3RhY2siLCJhb2kiOiIiLCJhc3NldHMiOlt7Iml0ZW1fdHlwZSI6IiIsImFzc2V0X3R5cGUiOiIiLCJpdGVtX2lkIjoiIn1dLCJ1cmwiOiJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY29tcHV0ZS1vcmRlcnMtbGl2ZS80ZThlNDZlZS03ZDYyLTQyNWMtYjljMy0yYmM2YTQ4YTljMzYvUFNTY2VuZTRCYW5kLzIwMjAwOTI1XzE2MTAyN180OF8yMjIzX21ldGFkYXRhLmpzb24_WC1Hb29nLUFsZ29yaXRobT1HT09HNC1SU0EtU0hBMjU2XHUwMDI2WC1Hb29nLUNyZWRlbnRpYWw9Y29tcHV0ZS1nY3Mtc3ZjYWNjJTQwcGxhbmV0LWNvbXB1dGUtcHJvZC5pYW0uZ3NlcnZpY2VhY2NvdW50LmNvbSUyRjIwMjIxMDIwJTJGYXV0byUyRnN0b3JhZ2UlMkZnb29nNF9yZXF1ZXN0XHUwMDI2WC1Hb29nLURhdGU9MjAyMjEwMjBUMTgzNjEyWlx1MDAyNlgtR29vZy1FeHBpcmVzPTg2Mzk5XHUwMDI2WC1H